#### [PyTorch를 활용한 머신러닝, 딥러닝 철저 입문]
## [Chapter 6-2] 예제: 와인 분류하기 (2)

---

## \#1. Import Modules

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

---

## \#2. Load Data

### 2-1. Get Wine Data

In [2]:
wine = load_wine()
print(wine.keys())

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])


---

## \#3. DataFrame

### 3-1. Create DataFrame in `df`

In [3]:
df = pd.DataFrame(wine.data, columns=wine.feature_names)
print(df.shape)
df.tail()

(178, 13)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.7,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.3,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.2,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.60,1.62,840.0
177,14.13,4.10,2.74,24.5,96.0,2.05,0.76,0.56,1.35,9.2,0.61,1.60,560.0


---

## \#4. Training / Test Data

### 4-1. Data Selecting

In [4]:
wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

`wine` 데이터에는 `class 0`, `class 1`, `class 2` 의 세 가지 데이터가 있지만, 이번 예제에서는 0과 1 두 가지 데이터만 사용한다.

In [5]:
wine.target[:130]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

총 178개의 데이터 중 index=130 까지의 데이터가 0과 1의 클래스를 가지는 데이터이므로, 이를 추출한다.

In [6]:
wine_data = wine.data[:130]
wine_target = wine.target[:130]
len(wine_data), len(wine_target)

(130, 130)

### 4-2. Get Train / Test data by `train_test_split`

위와 같은 방법 외에, `train_test_split` 함수를 이용해 데이터를 분리할 수 있다.

In [7]:
train_X, test_X, train_y, test_y = train_test_split(wine_data, wine_target, test_size=0.2)

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(104, 13) (104,)
(26, 13) (26,)


---

## \#5. Make Tensor

### 5-1. Transform Data Type

#### 완성된 train, test 데이터를 PyTorch의 `tensor 데이터`로 변환한다.

In [8]:
# train data
train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).long()

# test data
test_X = torch.from_numpy(test_X).float()
test_y = torch.from_numpy(test_y).long()

train_X.shape, train_y.shape

(torch.Size([104, 13]), torch.Size([104]))

#### train의 X와 y데이터를 `tensorDataset` 으로 합친다.

In [9]:
train = TensorDataset(train_X, train_y)
train[0]

(tensor([1.3050e+01, 1.6500e+00, 2.5500e+00, 1.8000e+01, 9.8000e+01, 2.4500e+00,
         2.4300e+00, 2.9000e-01, 1.4400e+00, 4.2500e+00, 1.1200e+00, 2.5100e+00,
         1.1050e+03]), tensor(0))

### 5-2. Mini Batch

#### train 데이터를 미니배치로 학습시킬 수 있도록 16개 단위로 분할한다.

In [10]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)

---

## \#6. Neural Network Composition

### 6-1. Neural Network class

이번에는 중간층이 총 5층인 신경망을 구성한다.
- 입력층의 노드 수 : 13개 [`x`의 feature 수]
- 중간층의 노드 수 : 각 96개
- 출력층의 노드 수 : 2개 [`y`의 `class_0`, `class_1`]

In [11]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 입력층
        self.func1 = nn.Linear(13, 96)
        
        # 중간층(4층))
        self.func2 = nn.Linear(96, 96)
        self.func3 = nn.Linear(96, 96)
        self.func4 = nn.Linear(96, 96)
        self.func5 = nn.Linear(96, 96)
        
        # 출력층
        self.func6 = nn.Linear(96, 2)
        
        
    def forward(self, x):
        # 입력층 ~ 중간층(4층) : 활성화함수는 relu 함수 사용
        x = F.relu(self.func1(x))
        x = F.relu(self.func2(x))
        x = F.relu(self.func3(x))
        x = F.relu(self.func4(x))
        x = F.relu(self.func5(x))
        
        # 출력층
        x = self.func6(x)
        
        # 출력층 활성화함수 : log_softmax
        y = F.log_softmax(x, dim=1)
        
        return y

### \# Jeina's comment

#### 위의 신경망 구성에서, `forward` 함수의 출력함수 (2층 활성화함수)로 사용한 log_softmax 함수에 대해 발생한 issue    


- 책에 나온 코드는 `y = F.log_softmax(x)` 로, `dim` 파라미터는 사용하지 않았다.


- 그러나 이 코드를 그대로 실행하면 아래의 에러가 발생한다.
```
UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
```


- `log_softmax` 함수의 `dim` 파라미터는 softmax 함수의 결과를 **어느 축을 기준으로 더해서 1이 되는 결과를 만들건지**를 결정한다.

> - `dim=0` : column 기준 (axis=0) 으로 모든 값을 더하면 1
- `dim=1` : row 기준 (axis=1) 으로 모든 값을 더하면 1

- 현재 와인 분류 문제에서는 category class가 0과 1로, 두 가지 클래스를 분류하는 문제


- 이 결과는 row 별로 결과가 출력되므로 (row의 길이가 2) `dim=1`이라는 파라미터를 추가했다.


- [Reference | PyTorch Discuss](https://discuss.pytorch.org/t/implicit-dimension-choice-for-softmax-warning/12314)

### 6-2. NN Instance 생성

In [12]:
model = Net()
model

Net(
  (func1): Linear(in_features=13, out_features=96, bias=True)
  (func2): Linear(in_features=96, out_features=96, bias=True)
  (func3): Linear(in_features=96, out_features=96, bias=True)
  (func4): Linear(in_features=96, out_features=96, bias=True)
  (func5): Linear(in_features=96, out_features=96, bias=True)
  (func6): Linear(in_features=96, out_features=2, bias=True)
)

입력층과 중간층 4층, 그리고 출력층까지 확인

---

## \#7. Model Training

### 7-1. Loss Function

In [13]:
criterion = nn.CrossEntropyLoss()

### 7-2. Optimizer

In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

### 7-3. Training

In [15]:
%%time
print("{} \t {}".format("epoch", "total loss"))
print("----- \t ----------------------")

# 7 iteration = 1 epoch
# 350 iteration = 50 epoch
for i in range(350):
    total_loss = 0
    
    for train_X, train_y in train_loader:
        
        # 계산 그래프 구성
        train_X, train_y = Variable(train_X), Variable(train_y)
        
        # gradient 초기화
        optimizer.zero_grad()
        
        # forward 계산
        output = model.forward(train_X)
        
        # loss 계산
        loss = criterion(output, train_y)
        
        # 오차 역전파
        loss.backward()
        
        # gradient update
        optimizer.step()
        
        # 누적 오차 계산
        total_loss += loss.data
        
    # 10번째 epoch마다 loss 출력    
    if (i+1) % 70 == 0:
        print("{} \t {}".format((i+1)//7, total_loss))

epoch 	 total loss
----- 	 ----------------------
10 	 1.9747740030288696
20 	 1.530383586883545
30 	 1.5853711366653442
40 	 1.42000150680542
50 	 2.200725555419922
CPU times: user 1.59 s, sys: 28.4 ms, total: 1.62 s
Wall time: 1.61 s


> 다음은 와인분류하기 (1)의 결과다.

#### Chapter 5.5 와인분류하기 (1)
```
epoch 	 total loss
----- 	 ----------------------
50 	 4.8501176834106445   
100 	 4.8353095054626465   
150 	 4.842362403869629   
200 	 4.866007328033447   
250 	 4.835178375244141   
300 	 4.835207462310791   
```

층을 딱 한 층만 썼던 [와인분류하기 (1)] 에서, 딱 5층만 늘렸을 뿐인데 결과가 크게 차이나는 것을 알 수 있다.

---

## \#8. Accuracy

### 8-1. test data 로 accuracy 계산

In [16]:
# 계산 그래프 구성
test_X, test_y = Variable(test_X), Variable(test_y)

# 결과를 0 또는 1이 되도록 변환
result = torch.max(model(test_X).data, 1)[1]

# test_y 와 결과가 같은 예측값의 개수 (맞춘 개수) 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy

0.9230769230769231

정확도는 약 92.3% 정도로, 이 또한 [와인분류하기 (1)] 에서의 65.3% 보다 훨씬 향상되었다 !